<a href="https://colab.research.google.com/github/Bilal-Ahmad-5/AI-Legal-Assistant/blob/main/AI_Legal_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU  langchain langchain_core huggingface_hub kagglehub chromadb langchain-groq
!pip install -qU  langchain langchain_community langchain_huggingface langchain-chroma taipy
!pip install -qU  pinecone langchain_pinecone langserve fastapi uvicorn sse_starlette python-dotenv jq
!pip install -qU  chromadb gradio pypdf tiktoken sentence_transformers langgraph
!pip install -qU "langchain-chroma>=0.1.2"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
taipy-gui 4.1.0 requires python-dotenv<=1.0.1,>=1.0.0, but you have python-dotenv 1.1.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 13.2 MB/s eta 0:00:00


In [ ]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("us-gpo/united-states-code")

print("Path to dataset files:", path)

# Look at the downloaded files
files = os.listdir(path)
print("Files in dataset directory:", files)


file_paths = [os.path.join(path, file) for file in files]
print("Full paths to dataset files:", file_paths)

Path to dataset files: /kaggle/input/united-states-code
Files in dataset directory: ['schemaandcss', 'USLM-User-Guide.pdf', 'US_Code_2017.json']
Full paths to dataset files: ['/kaggle/input/united-states-code/schemaandcss', '/kaggle/input/united-states-code/USLM-User-Guide.pdf', '/kaggle/input/united-states-code/US_Code_2017.json']


In [ ]:
from langchain_community.document_loaders import JSONLoader, CSVLoader, PyPDFLoader

def data_loader(file_paths):
    all_documents = []
    for file_path in file_paths:
        if file_path.endswith('.csv'):
            loader = CSVLoader(file_path)
        elif file_path.endswith('.json'):
            loader = JSONLoader(file_path, jq_schema='.', text_content=False)
        elif file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        else:
            print(f"Unsupported file type: {file_path}")
            continue
        all_documents.extend(loader.load())
    return all_documents

In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# Data Splitter
chunk_size = 500
chunk_overlap = 50

def text_splitter(data):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap
    )
  split_text = text_splitter.split_documents(data)
  return split_text

In [ ]:
## Vector db
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

def vector_database(chunks):
    embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

    vector_store = Chroma(
        collection_name="example_collection",
        embedding_function=embedding_model,
    )
    # Add documents to vector store
    vector_store.add_documents(chunks)
    return vector_store

In [ ]:
## Retriever
def retriever(file):
    splits = data_loader(file)
    chunks = text_splitter(splits)
    vectordb = vector_database(chunks)
    retriever = vectordb.as_retriever()
    return retriever

Retriever = retriever(file_paths)

Unsupported file type: /kaggle/input/united-states-code/schemaandcss


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
from langchain_community.tools import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from google.colab import userdata
import os

os.environ["TAVILY_API_KEY"] = userdata.get("tavily_api_key")
api = userdata.get("langchai_api_key")
os.environ["LANGCHAIN_API_KEY"] = api
os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')


llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

system_prompt = (
        "You are an AI Legal Assistant which is trained on US Law Data"
        "Help people and Professional Lawyers to learn and acknowledge"
        "Use the following context to answer the question. "
        "If you don't know the answer, say you don't know. "
        "Always say 'thanks for asking!' at the end of the answer. "
        "If the question is not related to the context, politely respond that you are tuned to only answer questions that are related to the context. "
        "\n\n{context}"
    )
prompt = ChatPromptTemplate.from_messages(
    [
            ("system", system_prompt),
            ("human", "{question}"),
    ]
)

tool_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a web ressearch, search on internet what user ask."),
    ("user", "{query}"),
])

tavily = TavilySearchResults()
retriever_tool = create_retriever_tool(Retriever , "retriever_tool" , "You are an Agent tools that is responsible for getting relevant documents from the database ")


In [ ]:
retriever_tool.invoke("what sis US law about rape?")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


'United States Legislative Model (USLM)  User Guide \n7 \n \nIn all courts, tribunals, and public offices of the United States, at home or abroad, of the \nDistrict of Columbia, and of each State, Territory, or insular possession of the United \nStates[,t]he matter set forth in the edition of the Code of Laws of the United States \ncurrent at any time shall, together with the then current supplement, if any, establish\n\n4.2 ANOMALOUS STRUCTURES  \nTwo issues drive the wide variety of legislative structures found in the United States Code. \nFirst, the United States Code contains Federal law enacted over a period of centuries, and legislative \ndrafting styles evolve over time. To handle this issue, the data is converted using the XML schema \nsubclassing mechanism. \nSecond, some Federal statutes contain features that do not conform to any commonly accepted drafting\n\nUnited States Legislative Model (USLM)  User Guide \n6 \n \n1 BRIEF INTRODUCTION TO THE UNITED STATES CODE \nThe sche

In [54]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()


tools = [retriever_tool, tavily]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [55]:
# Run the agent
config = {"configurable": {"thread_id": "abc123"}}

# Define the file path (this would come from user input in a real application)
# For demonstration, let's set it to a variable that can be None
user_provided_file_path = "/content/USLM-User-Guide.pdf" # Set to None to simulate no file

file_text = ""
if user_provided_file_path:
    try:
        # Use PyPDFLoader for PDF files
        from langchain_community.document_loaders import PyPDFLoader
        loader = PyPDFLoader(user_provided_file_path)
        file_content = loader.load()
        # Assuming you want to use the text content from the loaded documents
        file_text = " ".join([doc.page_content for doc in file_content])
        print(f"Loaded content from {user_provided_file_path}")

    except Exception as e:
        file_text = f"Could not load file: {e}"
        print(file_text)

# Construct the input message
query_text = "what is us law for bullying?"
if file_text and not file_text.startswith("Could not load file"):
    input_content = f"{query_text} Also, consider the information in this document: {file_text}"
else:
    input_content = query_text
    if file_text.startswith("Could not load file"):
        input_content += f"\nNote: {file_text}" # Add error message if file loading failed


input_message = {
    "role": "user",
    "content": input_content,
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

Could not load file: File path /content/USLM-User-Guide.pdf is not a valid file or url
================================ Human Message =================================

what is us law for bullying?
Note: Could not load file: File path /content/USLM-User-Guide.pdf is not a valid file or url
================================== Ai Message ==================================

Bullying in the United States is primarily addressed at the state level, with each state having its own anti-bullying laws. There is no single federal law specifically targeting bullying. However, certain federal laws may apply in specific circumstances, such as:

1. **Title IX of the Education Amendments of 1972**: Prohibits sex-based discrimination and harassment, including bullying based on sex, in educational institutions receiving federal funding.

2. **Individuals with Disabilities Education Act (IDEA)**: Protects students with disabilities from bullying that may constitute discrimination or interfere with their e

In [ ]:
print(chat)

what is us law for bullying?
Bullying is not a single, specific crime under U.S. federal law, but it can fall under various legal categories depending on the nature of the behavior. In the United States, laws addressing bullying are primarily handled at the **state level**, with most states having specific laws or policies to address bullying in schools and other contexts. Additionally, federal laws may apply in cases where bullying involves **discrimination**, **harassment**, or **cyberbullying**. Below is an overview of how bullying is addressed under U.S. law:

---

### **Federal Laws Related to Bullying**
While there is no single federal law specifically titled "anti-bullying," several federal laws may apply to bullying behavior:

1. **Title IX of the Education Amendments of 1972**:
   - Applies to schools that receive federal funding.
   - Prohibits sex-based discrimination, including sexual harassment and bullying based on gender.
   - Schools must investigate and address bullyin

# Task
Create a user interface for the agent using Gradio.

## Install gradio

### Subtask:
Install the Gradio library to create the user interface.


**Reasoning**:
The subtask is to install the Gradio library. I will use pip to install the package.



In [51]:
def predict_with_file(query: str, file):
    """
    Generates a response from the agent based on the user's query and an optional file.

    Args:
        query: The user's input query string.
        file: The file path if a file is uploaded, or None if no file is uploaded.

    Returns:
        The content of the agent's response as a string.
    """
    file_text = ""
    if file is not None:
        try:
            # Assuming only PDF files are supported for now based on previous code
            loader = PyPDFLoader(file)
            file_content = loader.load()
            file_text = " ".join([doc.page_content for doc in file_content])
            print(f"Loaded content from {file}")
        except Exception as e:
            file_text = f"Could not load file: {e}"
            print(file_text)

    input_content = query
    if file_text and not file_text.startswith("Could not load file"):
        input_content = f"{query} Also, consider the information in this document: {file_text}"
    elif file_text.startswith("Could not load file"):
        input_content += f"\nNote: {file_text}" # Add error message if file loading failed


    input_message = {
        "role": "user",
        "content": input_content,
    }

    # Assuming 'agent_executor' and 'config' are defined in the previous cells
    response = agent_executor.invoke({"messages": [input_message]}, config)
    return response["messages"][-1].content

In [57]:
import gradio as gr

# Create the Gradio interface
interface = gr.Interface(
    fn=predict_with_file,  # Use the prediction function defined earlier
    inputs=[
        gr.Textbox(label="Enter your query"),
        gr.File(label="Upload a document (Optional)")
    ],
    outputs=gr.Markdown(label="Agent Response"), # Changed output to Markdown
    title=" AI Legal Assistant",
    description="Enter your legal query and optionally upload a document for the agent to consider."
)

# Launch the interface
interface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c89e3c4c126ed29d36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c89e3c4c126ed29d36.gradio.live
